### Algorithm(Logic) behind Invisible cloak
- Capturing the live feed of the person
- Breaking the feed into images(frames)
- On the frames, implementing image segmentation(image segmentation is used to differentiate the cloak from the rest of the image and also for super-imposing the background image on the frame)
- In simple words, the main logic is super-imposing 2 images

- Before the person enters into the feed, the feed must 1st capture the background so that it can super-impose
- if we don't show the backgound, the code wont work
- 2 to 3 seconds is enough for the camera to capture background

### Important points:
- Make sure to use a cloth of uniform color to avoid noises and change hue value, corresponding to color of the cloth
- the color of the cloth should not be matched with background for better results

### Coding Part
- Make sure to have python 3.0 or higher version installed

In [1]:
# Importing Required Librraries
import numpy as np
import cv2
import time #time library is used to collect the background before setting up the cloak

In [4]:
#creating a video capture object
cap = cv2.VideoCapture(0)#id'0' represents the default webcam.If there are multiple webcams then there are multiple id's.
time.sleep(3)#to set up camera, as initially the camera feed is dark
background=0#background that will be displayed when we put the cloak on

#loop for capturing the background
#30 iterations so that background can be captured multiple times, the best can be picked while we put the cloak on
for i in range(60):
    ret, background=cap.read()#read frames from the webcam
    #returns 2 values
    #ret is a boolean value that represents a frame is captured or not
    #background has the captured frame
while cap.isOpened():#capture the video until the person want to be captured in webcam
    ret,img=cap.read()
    if ret==False:
        break
    #HSV is preferred over RGB because in RGB, beacuse R,G,B are dependent on each other
    #In HSV the colors change slowly and steadily and it depends on saturation
    #saturation means darkness of color 
    
    #HSB is independent of colors, it only depends on brightness and saturation
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)#converts the image(RGB) to HSV format
    #HSV(hue saturation value) is also known as HSB(B means Brightness)
    
    #In CV2, the hue values are reduced from 0-359 to 0-179 as cv2 can hold only 8-bits for color which is atmost 255
    #the 0-179 values are divided into more variations
    #In opencv the hue values are considered from 0-10 and 170-180 for red color cloak, as after 10 it starts fading out i.e., the values of another color starts
    lower_blue=np.array([100,120,70])#HSB values->below 120-saturation and 70-brightness these are really low in which frame would be entirely in white or red or pink etc. colors
    upper_blue=np.array([140,255,255])#hue value above 10 gives white etc.. which will not be efficient for the cloak
    mask1=cv2.inRange(hsv,lower_blue,upper_blue)#retreiving the captured frames which are in our required range i.e., seperating the cloak part
    
    #also our required frames also has hue values from 170-180, creating another array to capture frames in 170-180 range.
    lower_red=np.array([0,120,70])
    upper_red=np.array([10,255,255])#max value for brightness and saturation is 255,255
    mask2=cv2.inRange(hsv,lower_red,upper_red)
    
    mask1=mask1+mask2#operator overloading , it performs bitwise or operation
    #the above line retreieves the cloak part from our captured frames
    
    #morphology is used to remove noise like unnecessary pixels from the frames
    mask1=cv2.morphologyEx(mask1,cv2.MORPH_OPEN,np.ones((3,3),np.uint8),iterations=2)#removes noise from 9 pixels at a time
    mask1=cv2.morphologyEx(mask1,cv2.MORPH_DILATE,np.ones((3,3),np.uint8),iterations=2)#Dilates(smoothens the picture 9 pixels at a time)
    
    mask2=cv2.bitwise_not(mask1)#applying bitwise not operation between mask2 and mask1
    #It is used to retrieve the whole image except the cloak part
    
    #applying bitwise and operator between background and mask1
    #background is the image that have to be displayed when we have the cloak over 
    res1=cv2.bitwise_and(background,background,mask=mask1)#Used for Segmentation of the color i.e., differentiating cloak from the rest of the background
    #applying bitwise and operator between background and mask2
    #img is the image when we are infront of the camera
    res2=cv2.bitwise_and(img,img,mask=mask2)#Used to Substitute the Cloak part
    final_output=cv2.addWeighted(res1,1,res2,1,0)#super-imposing 2 images
    
    cv2.imshow("Abracadabra",final_output)
    k=cv2.waitKey(10)
    if k==27: #when escape key is pressed, the program stops
        break
cap.release()#release the object
cv2.destroyAllWindows()#destroy(close) all the windows(means stop running the video capture)